In [1]:
%matplotlib inline  
import pandas as pd
import matplotlib.pyplot as plt
import json 

In [2]:
df = pd.read_json('jobs.json')

In [3]:
len(df)

199582

In [5]:
df.head()

,jobtitle,company,city,state,country,language,formattedLocation,source,date,snippet,...,jobkey,sponsored,expired,indeedApply,formattedLocationFull,formattedRelativeTime,stations,refNum,keywords,advertiserId
0,"Bioinformatics Software Engineer, Production",Invitae,San Francisco,CA,US,en,"San Francisco, CA",InVitae,2021-03-31 00:41:21+00:00,Experience with Javascript frameworks such as ...,...,6fe3246003f0a2be,False,False,True,"San Francisco, CA 94107",8 days ago,,3029156,"[docker, javascript, bash]",NaN
1,iOS Lead,Rdio,San Francisco,CA,US,en,"San Francisco, CA",Rdio,2018-11-05 07:32:51+00:00,"Python, Java, Objective-C, JavaScript and Erla...",...,96407f75975a6cb0,False,False,True,"San Francisco, CA",2 years ago,,JL8193381111,"[swift, ios, android, java, javascript]",NaN
2,Senior Software Engineer II (Fullstack) - Vouc...,Uber,San Francisco,CA,US,en,"San Francisco, CA",Uber,2021-03-16 08:18:59+00:00,"Javascript, C++, Java, Python, or Go. Uber Vou...",...,23f9b7a73d200fbc,False,False,False,"San Francisco, CA 94103",22 days ago,,101262,"[node.js, c++, css, golang, angular, java, jav...",NaN
3,Senior Full Stack Developer,IBM,San Francisco,CA,US,en,"San Francisco, CA",IBM,2021-03-25 17:42:25+00:00,Software Developers at IBM are the backbone of...,...,a971331608ecbb66,False,False,False,"San Francisco, CA",13 days ago,,81488419,"[node.js, c++, angular, java, nosql, javascript]",NaN
4,Machine Learning Developer,The Blue Store,San Francisco,CA,US,en,"San Francisco, CA",Indeed,2021-02-12 00:00:01+00:00,"Use Python to design, implement and test autom...",...,61e0fa40e15cda82,False,False,True,"San Francisco, CA 94103",30+ days ago,,42583631-41-41,"[node.js, c++, c#, spark, tensorflow, java, ja...",42583631.0


In [8]:
df.groupby('city').jobtitle.count().reset_index(name='count').sort_values(['count'], ascending=False).head(10)

,city,count
2148,New York,9279
2828,Seattle,6157
2748,San Francisco,6023
136,Austin,5469
529,Chicago,5294
122,Atlanta,4246
292,Boston,3964
694,Dallas,3313
3306,Washington,3147
1755,Los Angeles,2965


In [9]:
df.groupby('state').jobtitle.count().reset_index(name='count').sort_values(['count'], ascending=False).head(10)

,state,count
5,CA,36097
44,TX,17858
35,NY,13685
47,WA,10489
46,VA,10114
21,MA,9034
16,IL,8289
11,FL,8002
22,MD,7439
28,NC,6535


In [11]:
df.groupby('company').jobtitle.count().reset_index(name='count').sort_values(['count'], ascending=False).head(10)

,company,count
8987,Deloitte,4380
1886,Amazon.com Services LLC,4246
1885,"Amazon Web Services, Inc.",2953
16175,KPMG,1852
1876,"Amazon Dev Center U.S., Inc.",1381
31593,VMware,1236
19047,Microsoft,1235
8586,DaVita,1180
11081,Facebook,1131
27409,Splunk,943


In [12]:
df.groupby('jobtitle').jobtitle.count().reset_index(name='count').sort_values(['count'], ascending=False).head(10)

,jobtitle,count
80332,Software Engineer,2818
72235,Senior Software Engineer,2052
78373,Software Developer,1027
17755,Data Engineer,834
20517,DevOps Engineer,826
51103,Pharmacy Technician,739
18727,Data Scientist,679
95379,Systems Administrator,644
37947,Java Developer,585
29759,Full Stack Developer,531


In [17]:
df['keywords']

0                                [docker, javascript, bash]
1                   [swift, ios, android, java, javascript]
2         [node.js, c++, css, golang, angular, java, jav...
3          [node.js, c++, angular, java, nosql, javascript]
4         [node.js, c++, c#, spark, tensorflow, java, ja...
                                ...                        
199577                                         [tensorflow]
199578                                          [expressjs]
199579                                          [expressjs]
199580                                          [expressjs]
199581                                          [expressjs]
Name: keywords, Length: 199582, dtype: object

In [37]:
keyword_dict = {}
def count_keywords(keyword):

    for item in keyword:
        if item in keyword_dict:
            keyword_dict[item] += 1
        else:
            keyword_dict[item] =1

In [41]:
df.keywords.map(count_keywords)
keyword_dict = dict(sorted(keyword_dict.items(), key=lambda item: item[1], reverse=True))
print(keyword_dict)

{'python': 174980, 'java': 148816, 'linux': 148060, 'javascript': 145604, 'aws': 133616, 'azure': 118696, 'c++': 97868, 'html': 96680, 'css': 94112, 'c#': 93208, 'spark': 75508, 'docker': 72104, 'mysql': 48312, 'ruby': 46540, 'angular': 46244, 'nosql': 43700, 'perl': 39020, 'android': 37336, 'bash': 37036, 'hadoop': 36532, 'c': 36520, 'node.js': 31676, 'php': 31620, 'asp.net': 24124, 'jquery': 22968, 'scala': 21100, 'vba': 15336, 'tensorflow': 15112, 'vue': 14480, 'r': 9392, 'kotlin': 9220, 'golang': 8764, 'reactjs': 7920, 'swift': 7904, 'rust': 7736, 'django': 6628, 'mongo': 6436, 'ios': 4528, 'laravel': 2784, 'clojure': 1536, 'haskell': 1412, 'lisp': 464, 'expressjs': 440}


In [33]:
keyword_dict['docker']

18026

In [46]:
df['']

KeyError: False

0.0

False